[Reference](https://microprediction.medium.com/combining-pycaret-and-timemachines-for-time-series-prediction-a4d456e47cd9)

In [3]:
!pip install pycaret[full]
!pip install --upgrade statsmodels
!pip install microprediction
!pip install timemachines
import microprediction
from datetime import datetime, timedelta
from microprediction import MicroReader
import random
import matplotlib.pyplot as plt
import pandas as pd

     |████████████████████████████████| 14.8 MB 32.0 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
microprediction 0.18.4 requires numpy>=1.20.1, but you have numpy 1.19.5 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.6.0 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.30.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
copulas 0.6.0 requires numpy<2,>=1.20.0; python_version >= "3.7", but you have numpy 1.19.5 which is incompatible.
albumentati

  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
pycaret 2.3.5 requires numpy==1.19.5, but you have numpy 1.21.4 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.6.0 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.30.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albument

ContextualVersionConflict: ignored

In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 27.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
mr = MicroReader()
all_streams = mr.get_stream_names()
lagged_values = []
while len(lagged_values) < 900:
  a_stream = random.choice(all_streams)
  lagged_values, lagged_seconds = mr.get_lagged_values_and_times(a_stream)
values = list(reversed(lagged_values))
dt = [ datetime.fromtimestamp(s) for s in reversed(lagged_seconds)] 
plt.plot(dt,values)
plt.title(a_stream)

In [ ]:
df = pd.DataFrame(columns=['Date','y'])
df['date'] = dt
df['y']=values
df['dayofweek'] = df['date'].dt.dayofweek
df['hour'] = df['date'].dt.hour
num_lags = 10
lags = range(1,num_lags)
lag_names = [ 'y_'+str(lag) for lag in lags ]
for lag, lag_name in zip(lags,lag_names):
  df[lag_name] = df['y'].shift(lag)
numerical_features = lag_names
categorical_features =  ['dayofweek','hour']

In [ ]:
s = setup(df, target = 'y', train_size = 0.95,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['date'],
              numeric_features = numerical_features,
              categorical_features = categorical_features,
              silent = True, verbose = False, session_id = 123)
top5 = compare_models(n_select = 5)

In [ ]:
from timemachines.skaters.allskaters import EMA_SKATERS, DLM_SKATERS, THINKING_SKATERS, TSA_SKATERS, HYPOCRATIC_ENSEMBLE_SKATERS
from timemachines.skating import prior 
skaters = EMA_SKATERS + DLM_SKATERS + TSA_SKATERS + HYPOCRATIC_ENSEMBLE_SKATERS
skater_names = [ f.__name__ for f in skaters ]
for f, skater_name in zip(skaters,skater_names):
    print('Running '+skater_name)
    y = df['y'].values
    x,x_std = prior(f, y=y, k=1) # Runs a time-series model forward
    df[skater_name] = x

In [ ]:
s = setup(df, target = 'y', train_size = 0.95,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['date'],
              numeric_features = numerical_features + skater_names,
              categorical_features = categorical_features,
              silent = True, verbose = False, session_id = 123)
top5again = compare_models(n_select = 5)